# Neighbors and districts pre-processing.
<a id='top_cell'></a>
Following the Preprocessing and filtering used on (Preprocess and EDA.ipynb), we generate an extra pre-process to be able to complete the missing information and standardized Locations data such as Neighbors and Districts (Barrios y Comunas)

This Notebook was originally on Colab.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tilemapbase
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tilemapbase: filename=tilemapbase-0.4.7-py3-none-any.whl size=25790 sha256=353cdaff3916b940947fe1b09350f98763c53c556d432d5bac12c4a197aca29a
  Stored in directory: /root/.cache/pip/wheels/bd/d5/68/efc210bd62a488203c75d4ba1d61a0039bc799772a93ce2ae3
Successfully built tilemapbase
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 8.1 MB/s 
     |████████████████████████████████| 16.7 MB 48.0 MB/s 
     |████████████████████████████████| 6.3 MB 51.6 MB/s 


In [11]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point,Polygon
import matplotlib.pyplot as plt
import datetime
import os
from tqdm import tqdm
import numpy as np
import shapely
from operator import itemgetter

##Datasets
First is nesesary 3 datasets, the original Dataset that can be rather Crimes or Resource dataset filter from the last Notebook, and 2 ShapeFiles that contained the information of Neighbors and districts available on https://data.metabolismofcities.org/dashboards/medellin/

In [13]:
data = pd.read_csv('/content/drive/MyDrive/DS4A/dataframes/df_crimenes_2.csv')
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,19,20,21,22,29,30,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,fecha_hecho,cantidad,latitud,longitud,sexo,edad,estado_civil,grupo_actor,actividad_delictiva,...,categoria_bien,grupo_bien,modelo,color,permiso,unidad_medida,fecha_ingestion,Nbase,fecha,hora
0,0,2003-01-01t10:00:00.000-05:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-06t04:05:34.000-05:00,extorsion,2003-01-01,10:00:00.000-05:00
1,1,2003-01-01t11:00:00.000-05:00,1.0,NaN,NaN,NaN,NaN,NaN,guerrilla,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-06t04:05:34.000-05:00,extorsion,2003-01-01,11:00:00.000-05:00
2,2,2003-01-10t06:00:00.000-05:00,1.0,NaN,NaN,hombre,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-06t04:05:34.000-05:00,extorsion,2003-01-10,06:00:00.000-05:00
3,3,2003-01-11t10:00:00.000-05:00,1.0,NaN,NaN,hombre,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-06t04:05:34.000-05:00,extorsion,2003-01-11,10:00:00.000-05:00
4,4,2003-01-15t09:00:00.000-05:00,1.0,NaN,NaN,mujer,37.0,NaN,delincuencia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-06t04:05:34.000-05:00,extorsion,2003-01-15,09:00:00.000-05:00


In [14]:
map = gpd.read_file("/content/drive/MyDrive/DS4A/EDA/Mapas/BarrioVereda_2014.shp")
map.head()

,CODIGO,NOMBRE,ABREVIATUR,IDENTIFICA,LIMITECOMU,ZHFISICAGE,LIMITEMUNI,VIGENCIA_F,SUBTIPO_BA,LINK_DOCUM,geometry
0,Inst_15,U.P.B,None,Inst_15,11,None,001,None,1,None,"POLYGON ((-75.58911 6.24447, -75.58900 6.24443..."
1,Inst_18,Cerro Nutibara,None,Inst_18,16,None,001,None,1,None,"POLYGON ((-75.57669 6.23733, -75.57628 6.23612..."
2,0610,Mirador del Doce,None,0610,06,None,001,None,1,None,"POLYGON ((-75.58450 6.30520, -75.58445 6.30519..."
3,1620,El Nogal-Los Almendros,None,1620,16,None,001,None,1,None,"POLYGON ((-75.59551 6.23662, -75.59552 6.23660..."
4,0612,El Triunfo,None,0612,06,None,001,None,1,None,"POLYGON ((-75.58618 6.31007, -75.58616 6.31007..."


In [15]:
map2 = gpd.read_file("/content/drive/MyDrive/DS4A/EDA/Mapas/LimiteComunaCorregimiento_2014.shp")
map2.head()

,OBJECTID,CODIGO,NOMBRE,IDENTIFICA,LIMITEMUNI,SUBTIPO_CO,LINK_DOCUM,geometry
0,1,01,Popular,Comuna 1,001,1,None,"POLYGON ((-75.54803 6.30436, -75.54795 6.30431..."
1,2,02,Santa Cruz,Comuna 2,001,1,None,"POLYGON ((-75.55703 6.31007, -75.55699 6.31003..."
2,3,03,Manrique,Comuna 3,001,1,None,"POLYGON ((-75.54252 6.28632, -75.54252 6.28632..."
3,4,04,Aranjuez,Comuna 4,001,1,None,"POLYGON ((-75.56229 6.29126, -75.56221 6.29125..."
4,5,05,Castilla,Comuna 5,001,1,None,"POLYGON ((-75.56566 6.31261, -75.56559 6.31259..."


In [ ]:
crimenes = data['Nbase'].unique()
crimenes

array(['aprehension', 'captura',
       'conduccion para el restablecimiento de derechos',
       'conducta contraria a la convivencia', 'incautacion arma de fuego',
       'incautacion mercancia',
       'lesion fatal accidente de transito segun policia',
       'lesion fatal accidente de transito segun policia con precipitacion',
       'lesion no fatal accidental segun policia',
       'lesion no fatal accidente de transito segun policia',
       'lesion no fatal dolosa segun policia', 'ley1098',
       'llamada nuse123', 'recuperacion de automotores',
       'recuperacion de mercancia',
       'solicitud de medidas de proteccion por violencia intrafamiliar'],
      dtype=object)

##Check Neighbors with Polygons
Using the polygons and names from the map dataframe from the Shapefile of Neighbors using contain funtion and we replace with map name to standarize the locations with a correct latitude and longitude coordinates

In [16]:
barrios = map['geometry'].reset_index(drop=True)
barrios

0      POLYGON ((-75.58911 6.24447, -75.58900 6.24443...
1      POLYGON ((-75.57669 6.23733, -75.57628 6.23612...
2      POLYGON ((-75.58450 6.30520, -75.58445 6.30519...
3      POLYGON ((-75.59551 6.23662, -75.59552 6.23660...
4      POLYGON ((-75.58618 6.31007, -75.58616 6.31007...
                             ...                        
327    POLYGON ((-75.61219 6.22300, -75.61219 6.22299...
328    POLYGON ((-75.60961 6.21869, -75.60961 6.21869...
329    POLYGON ((-75.63648 6.28070, -75.63648 6.28070...
330    POLYGON ((-75.61056 6.21853, -75.61056 6.21853...
331    POLYGON ((-75.58111 6.19704, -75.58162 6.19666...
Name: geometry, Length: 332, dtype: geometry

In [ ]:
for i in tqdm(range(len(data))):
  for ind in range(len(barrios)): 
    point = Point(data['longitud'][i],data['latitud'][i])
    isT = barrios[ind].contains(point)
    if isT == True:
      data['nombre_barrio'][i] = map['NOMBRE'][ind]

  0%|          | 0/1444550 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 1444550/1444550 [7:08:17<00:00, 56.21it/s]


Using colab implies that time is limited, so we use save points on the dataset after every large action 

In [ ]:
#data.to_csv('/content/drive/MyDrive/DS4A/dataframes/Crimen_Barrios_Comunas_Cor_2.csv')

##Check Districts with Polygons
Using the polygons and names from the map2 dataframe from the Shapefile of Districts using contain funtion and we replace with map2 name to standarize the locations with a correct latitude and longitude coordinates

In [17]:
comunas = map2['geometry'].reset_index(drop=True)
comunas

0     POLYGON ((-75.54803 6.30436, -75.54795 6.30431...
1     POLYGON ((-75.55703 6.31007, -75.55699 6.31003...
2     POLYGON ((-75.54252 6.28632, -75.54252 6.28632...
3     POLYGON ((-75.56229 6.29126, -75.56221 6.29125...
4     POLYGON ((-75.56566 6.31261, -75.56559 6.31259...
5     POLYGON ((-75.57586 6.31093, -75.57584 6.31092...
6     POLYGON ((-75.59050 6.29592, -75.59049 6.29590...
7     POLYGON ((-75.55077 6.26237, -75.55071 6.26237...
8     POLYGON ((-75.55502 6.24505, -75.55501 6.24504...
9     POLYGON ((-75.56426 6.26394, -75.56412 6.26393...
10    POLYGON ((-75.59147 6.26835, -75.59146 6.26834...
11    POLYGON ((-75.59530 6.27190, -75.59525 6.27176...
12    MULTIPOLYGON (((-75.61568 6.26992, -75.61569 6...
13    POLYGON ((-75.56919 6.22658, -75.56919 6.22649...
14    POLYGON ((-75.61221 6.23916, -75.61217 6.23906...
15    POLYGON ((-75.66691 6.37391, -75.66682 6.37354...
16    MULTIPOLYGON (((-75.64947 6.33105, -75.64944 6...
17    POLYGON ((-75.65450 6.26043, -75.65381 6.2

In [ ]:
for i in tqdm(range(len(data))):
  for ind in range(len(comunas)): 
    point = Point(data['longitud'][i],data['latitud'][i])
    isT = comunas[ind].contains(point)
    if isT == True:
      data['codigo_comuna'][i] = map2['IDENTIFICA'][ind]

  0%|          | 0/1444550 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 1444550/1444550 [2:45:40<00:00, 145.32it/s]


In [ ]:
#data.to_csv('/content/drive/MyDrive/DS4A/dataframes/Crimen_Barrios_Comunas_Cor_3.csv')

##Check Districts and Neighbors with Names
Using the names of both shapefile dataframe and the difflib library we correct most of the names of neighbors and districts

In [ ]:
data['nombre_barrio'] = data['nombre_barrio'].str.replace('sin dato','')

In [ ]:
barNull = []
comNull = []
barName = []
comName = []
for i in tqdm(range(len(data))):
  if pd.isna(data['latitud'][i]):
    if pd.isna(data['nombre_barrio'][i]):
      barNull.append(i)
    else:
      barName.append(data['nombre_barrio'][i])
    if pd.isna(data['codigo_comuna'][i]):
      comNull.append(i)
    else:
      comName.append(data['codigo_comuna'][i])

100%|██████████| 493568/493568 [00:04<00:00, 105842.09it/s]


In [ ]:
np.unique(barName)

array([' 13 san javier', ' 16 belen', ' 3 manrique', ' 50 palmitas',
       ' 90 santa elena', 'Aguas Frias', 'Aldea Pablo VI',
       'Alejandro Echavarría', 'Alejandría', 'Alfonso López', 'Altamira',
       'Altavista', 'Altos del Poblado', 'Andalucía', 'Antonio Nariño',
       'Aranjuez', 'Asomadera No.1', 'Astorga', 'Aures No.1',
       'Aures No.2', 'B. Cerro El Volador', 'Barrio Caycedo',
       'Barrio Colombia', 'Barrio Colón', 'Barrio Cristóbal',
       'Barrios de Jesús', 'Barro Blanco', 'Batallón Girardot',
       'Belalcázar', 'Belencito', 'Bello Horizonte', 'Belén', 'Berlin',
       'Bermejal-Los Alamos', 'Betania', 'Blanquizal', 'Bolivariana',
       'Bomboná No.1', 'Bomboná No.2', 'Boquerón', 'Bosques de San Pablo',
       'Boston', 'Boyacá', 'Brasilia', 'Buenos Aires',
       'Cabecera Urbana Corregimiento San Cristóbal', 'Calasanz',
       'Calasanz Parte Alta', 'Calle Nueva', 'Campo Alegre', 'Campo Amor',
       'Campo Valdés No.1', 'Campo Valdés No.2', 'Caribe',
    

In [ ]:
np.unique(comName)

array(['10', '13', '14', '15', '16', '3', '4', '5', '50', '60', '70',
       '80', '90', 'Comuna 1', 'Comuna 10', 'Comuna 11', 'Comuna 12',
       'Comuna 13', 'Comuna 14', 'Comuna 15', 'Comuna 16', 'Comuna 2',
       'Comuna 3', 'Comuna 4', 'Comuna 5', 'Comuna 6', 'Comuna 7',
       'Comuna 8', 'Comuna 9', 'Corregimiento 50', 'Corregimiento 60',
       'Corregimiento 70', 'Corregimiento 80', 'Corregimiento 90'],
      dtype='<U16')

In [ ]:
for i in tqdm(range(len(data))):
  if pd.isna(data['codigo_comuna'][i]) == False:
    if data['codigo_comuna'][i] == 'SIN DATO':
      data['codigo_comuna'][i] = np.nan
    else:
      st = str(data['codigo_comuna'][i])
      if len(st) <= 2:
        num = int(data['codigo_comuna'][i])
        if num <= 16:
          data['codigo_comuna'][i] = "Comuna "+str(num)
        else:
          data['codigo_comuna'][i] = "Corregimiento "+str(num)

  0%|          | 0/493568 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
  6%|▋         | 31139/493568 [00:00<00:11, 40989.63it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
100%|██████████| 493568/493568 [00:16<00:00, 30106.93it/s]


In [ ]:
data['codigo_comuna'].unique()

array([nan, 'Comuna 3', 'Comuna 8', 'Comuna 10', 'Comuna 16', 'Comuna 13',
       'Comuna 4', 'Comuna 1', 'Comuna 2', 'Comuna 7', 'Comuna 12',
       'Comuna 11', 'Corregimiento 60', 'Comuna 14', 'Comuna 6',
       'Comuna 5', 'Comuna 15', 'Comuna 9', 'Corregimiento 50',
       'Corregimiento 80', 'Corregimiento 70', 'Corregimiento 90'],
      dtype=object)

In [ ]:
import difflib

In [ ]:
real=[]
actual =[]
names = data['nombre_barrio'].dropna().unique()
for i in names:
  a = difflib.get_close_matches(i,map['NOMBRE'].unique() , n=10, cutoff=0.5)
  if len(a)>0:
    if i != a[0]:
      print(a)
      real.append(a[0])
      actual.append(i)

['San Javier No.2', 'San Javier No.1']
['Palenque', 'Manrique Oriental']
['Santa Teresita', 'Santa Mónica']
['San José', 'Los Naranjos', 'La Rosa', 'Las Granjas', 'Aranjuez']
['Las Palmas', 'La Palma']


In [ ]:
print(actual)

[' 13 san javier', ' 3 manrique', ' 90 santa elena', 'san jose', ' 50 palmitas']


In [ ]:
for i in tqdm(range(len(data))):
  if data['nombre_barrio'][i] in actual:
    for j in range(len(actual)):
      if data['nombre_barrio'][i] == actual[j]:
        data['nombre_barrio'][i] = real[j]

  0%|          | 0/493568 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 493568/493568 [00:03<00:00, 146858.32it/s]


In [ ]:
data['nombre_barrio'].unique()

array([nan, 'Campo Valdés No.2', 'Las Estancias', 'Villa Nueva',
       'El Nogal-Los Almendros', 'El Pesebre', 'Belén', 'Moravia',
       'Barrio Colón', 'Prado', 'Estación Villa', 'Granizal', 'Andalucía',
       'Popular', 'San Diego', 'Robledo', 'La Candelaria',
       'Simón Bolívar', 'Laureles', 'Santa Mónica', 'Villa del Socorro',
       'Villa Hermosa', 'Veinte de Julio',
       'Cabecera Urbana Corregimiento San Cristóbal',
       'Manrique Central No.1', 'Boston', 'El Poblado',
       'Doce de Octubre No.2', 'Santo Domingo Savio No.1', 'San Antonio',
       'Campo Valdés No.1', 'San Lucas', 'Suramericana', 'Castropol',
       'Las Brisas', 'Bomboná No.1', 'Villa Niza', 'Fátima',
       'Jesús Nazareno', 'Pablo VI', 'La América',
       'Terminal de Transporte', 'La Colina', 'Bolivariana', 'Trinidad',
       'Altos del Poblado', 'Pedregal Alto', 'La Aguacatala',
       'Los Alcázares', 'Carlos E. Restrepo', 'Sin Nombre',
       'Perpetuo Socorro', 'Florida Nueva', 'La Esperanza

In [ ]:
difflib.get_close_matches( ' 16 belen',map['NOMBRE'].unique() , n=10, cutoff=0.3)

['Belencito',
 'Belén',
 'Los Ángeles',
 'Berlin',
 'Robledo',
 'La Castellana',
 'Palenque',
 'Laureles',
 'La Verde',
 'La Salle']

In [ ]:
data['nombre_barrio'] = data['nombre_barrio'].replace({' 11 laureles -  estadio': 'Laureles', ' - envigado': np.nan,'u.p.b':'U.P.B','el llano santa elena':np.nan,' 16 belen':'Belén'})

In [ ]:
#data.to_csv('/content/drive/MyDrive/DS4A/dataframes/Crimen_Barrios_Comunas_Cor_3.csv', index=False)

##Check Districts with Neighbors polygon
Using the polygons of both shapefile dataframe we compare if a Neighbor is inside a District to fill the blank districts in which the Neighbor is available

In [ ]:
for i in tqdm(range(len(data))):
  geo = map[(map['NOMBRE']==data['nombre_barrio'][i]).values]['geometry']
  if len(geo) > 0:
    for ind in range(len(comunas)):
      isT = comunas[ind].contains(geo.values[0])
      if isT == True:
        data['codigo_comuna'][i] = map2['IDENTIFICA'][ind]

Streaming output truncated to the last 5000 lines.
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
100%|█████████▉| 492397/493568 [1:46:16<00:17, 68.38it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher

In [ ]:
data.to_csv("/content/drive/MyDrive/DS4A/dataframes/Crimen_Barrios_Comunas_Cor_7.csv", index=False)